In [1]:
from vyper.user import Model
import pandas as pd
from sklearn.utils import shuffle
from vyper.user.explorer import DataProfiler
from openpyxl import Workbook

import math
import numpy as np
from vyper.utils.tools import StatisticalTools as st
from sklearn.preprocessing import OrdinalEncoder
import scipy as stats
from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler
from varclushi import VarClusHi
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [2]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from k_means_constrained import KMeansConstrained
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from kneed import KneeLocator
from yellowbrick.cluster import KElbowVisualizer
from matplotlib.cm import viridis
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from joblib import Parallel, delayed
import numpy as np
import multiprocessing
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
class Hierarchical:
    def __init__(self, data):
        self.df = pd.read_csv(data)
        # self.df = self.df.iloc[:20000, :]

    def get_shape(self):
        return self.df.shape

    def get_df(self):
        return self.df

    def dendogram(self):
        plt.figure(figsize=(10, 7))
        plt.title("Dendogram")
        shc.dendrogram(shc.linkage(self.df, method='ward'))
        plt.show()

    @staticmethod
    def silhouette(estimator, df, metric='euclidean'):
        # print(self.df.shape)
        labels = estimator.fit_predict(df)
        score = silhouette_score(df, labels, metric=metric)
        return score

    # def create_folds(self, n):
    #     df_folds = self.df.copy()
    #     df_folds["fold"] = np.random.randint(1, n + 1, size=len(self.df))
    #     return df_folds

    # def nfold(self, folded_data, estimator, n):
    #     #randomly assign folds to the df
    #     # df_folds = self.df.copy()
    #     # df_folds["fold"] = np.random.randint(1, n + 1, size=len(self.df))
    #     score = 0
    #     #select one fold and use the rest folds as input for modeling
    #     for i in range(1, n + 1):
    #         # self.df = df_folds[df_folds.fold != i].copy()
    #         # self.df.drop("fold", axis=1, inplace=True)
    #         sil = self.silhouette(estimator, folded_data[folded_data.fold != i].drop(['fold'], axis=1),
    #                               metric='euclidean')
    #         score += sil
    #     score = score / n
    #     return score

    # def gridSearchCV_hierarchical(self, grid, cv=5):
    #     score_max = 0
    #     linkage = n_clusters = affinity = None
    #     fd = self.create_folds(cv)
    #     training_history = {'score':[], 'n_clusters': [], 'linkage': [], 'affinity': []}
    #     # self.training_history = {}
    #     for n in grid["n_clusters"]:
    #         for l in grid["linkage"]:
    #             for a in grid["affinity"]:
    #                 if l == "ward" and a != "euclidean":
    #                     continue
    #                 else:
    #                     hierarchical = AgglomerativeClustering(n_clusters=n, affinity=a, linkage=l)
    #                     score = self.nfold(fd, hierarchical, cv)
    #                     training_history['score'].append(score)
    #                     training_history['n_clusters'].append(n)
    #                     training_history['linkage'].append(l)
    #                     training_history['affinity'].append(a)
    #                     # print("score:", score, " n_clusters:", n, " linkage:", l, " affinity:", a)
    #                     if score > score_max:
    #                         score_max = score
    #                         linkage = l
    #                         affinity = a
    #                         n_clusters = n
    #     return score_max, n_clusters, linkage, affinity, training_history

    @staticmethod
    def get_training_history(training_history):
        return pd.DataFrame.from_dict(training_history)



    # def gridSearchCV_hierarchical(self, grid, cv=5, n_jobs=-1, verbose=0):
    #     score_max = 0
    #     linkage = n_clusters = affinity = None
    #     fd = self.create_folds(cv)
    #     training_history = {'score':[], 'n_clusters': [], 'linkage': [], 'affinity': []}
    #     param_combinations = [(n, l, a) for n in grid['n_clusters'] for l in grid['linkage'] for a in grid['affinity'] if not (l == 'ward' and a != 'euclidean')]
    #     if verbose:
    #         print(f'Number of combinations to test: {len(param_combinations)}')
    #     scores = Parallel(n_jobs=n_jobs)(delayed(self._fit_hierarchical)(n, l, a, fd,cv) for n, l, a in param_combinations)
    #     for i, score in enumerate(scores):
    #         n, l, a = param_combinations[i]
    #         training_history['score'].append(score)
    #         training_history['n_clusters'].append(n)
    #         training_history['linkage'].append(l)
    #         training_history['affinity'].append(a)
    #         if score > score_max:
    #             score_max = score
    #             linkage = l
    #             affinity = a
    #             n_clusters = n
    #     return score_max, n_clusters, linkage, affinity, training_history
    #
    # def _fit_hierarchical(self, n_clusters, linkage, affinity, fd,cv):
    #     hierarchical = AgglomerativeClustering(n_clusters=n_clusters, affinity=affinity, linkage=linkage)
    #     score = self.nfold(fd, hierarchical,cv)
    #     return score
    # def gridSearchCV_hierarchical(self, grid, cv=5):
    #     score_max = 0
    #     linkage = n_clusters = affinity = None
    #     fd = self.create_folds(cv)
    #     training_history = {'score':[], 'n_clusters': [], 'linkage': [], 'affinity': []}
    #
    #     def run_hierarchical(n, l, a):
    #         if l == "ward" and a != "euclidean":
    #             return None
    #         hierarchical = AgglomerativeClustering(n_clusters=n, affinity=a, linkage=l)
    #         score = self.nfold(fd, hierarchical, cv)
    #         return score, n, l, a
    #
    #     num_cores = multiprocessing.cpu_count()
    #     print("Number of cores:", num_cores)
    #     results = Parallel(n_jobs=num_cores)(
    #         delayed(run_hierarchical)(n, l, a) for n in grid["n_clusters"]
    #                                           for l in grid["linkage"]
    #                                           for a in grid["affinity"]
    #     )
    #
    #     for result in results:
    #         if result is not None and result[0] > score_max:
    #             score_max, n_clusters, linkage, affinity = result[0], result[1], result[2], result[3]
    #         if result is not None:
    #             training_history['score'].append(result[0])
    #             training_history['n_clusters'].append(result[1])
    #             training_history['linkage'].append(result[2])
    #             training_history['affinity'].append(result[3])
    #
    #     return score_max, n_clusters, linkage, affinity, training_history



    def gridSearchCV_hierarchical(self, grid, cv=5):
        # fd = self.create_folds(cv)
        hierarchical = AgglomerativeClustering()
        grid_search = GridSearchCV(estimator=hierarchical, param_grid=grid, cv=cv, scoring=self.silhouette)
        grid_search.fit(self.df)
        training_history = grid_search.cv_results_
        return grid_search.best_score_, grid_search.best_params_, training_history

    def randomizedSearchCV_hierarchical(self, grid, cv=5, n_iter=10,rand_sample_prop=0.2):
        # fd = self.create_folds(cv)
        sample_data = self.df.sample(frac=rand_sample_prop)
        hierarchical = AgglomerativeClustering()
        random_search = RandomizedSearchCV(estimator=hierarchical, param_distributions=grid, cv=cv, scoring=self.silhouette, n_iter=n_iter)
        random_search.fit(sample_data)
        training_history = random_search.cv_results_
        return random_search.best_score_, random_search.best_params_, training_history

In [4]:
folds = 5
params_grid = {"linkage": ["ward", "complete", "average", "single"],
               "n_clusters": list(range(2, 10)),
               "affinity": ["euclidean", "l1", "l2", "manhattan", "cosine"]}
# params_grid = {"linkage": ["complete", "average", "single"],
#                "n_clusters": list(range(2, 10)),
#                "affinity": ["euclidean", "manhattan", "cosine"]}

In [5]:
hi = Hierarchical('maurices_oc_preprocessed.csv')

In [6]:
hi.get_shape()

(99942, 13)

In [7]:
import time

start_time = time.time()
a, b, c = hi.randomizedSearchCV_hierarchical(params_grid, folds)
print("--- %s seconds ---" % (time.time() - start_time))

--- 333.2494750022888 seconds ---


In [16]:
score_max, n_clusters, linkage, affinity = a, b['n_clusters'], b['linkage'], b['affinity']

In [17]:
c

{'mean_fit_time': array([1.06222529e+01, 9.73192034e+00, 1.00706166e+01, 3.01735048e+00,
        2.81138420e-03, 2.28419304e-03, 1.57216191e+00, 1.00420602e+01,
        1.60499477e+00, 1.05856113e+01]),
 'std_fit_time': array([5.22299703e-01, 2.19574825e-01, 2.48395323e-01, 1.24097444e-01,
        7.46487671e-04, 3.93809517e-04, 2.51387340e-02, 7.53241747e-02,
        3.56634499e-02, 1.22780201e+00]),
 'mean_score_time': array([0.76795197, 0.67951565, 0.73575082, 0.45559478, 0.        ,
        0.        , 0.35424495, 0.72591538, 0.34792509, 0.82168465]),
 'std_score_time': array([0.04472238, 0.00942883, 0.05044203, 0.01786562, 0.        ,
        0.        , 0.00628541, 0.03478234, 0.00612616, 0.14825182]),
 'param_n_clusters': masked_array(data=[8, 7, 7, 3, 5, 3, 8, 2, 4, 2],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_linkage': masked_array(data=['complete',

In [18]:
a

0.7191736249351095

In [19]:
hi.get_training_history(c)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_clusters,param_linkage,param_affinity,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,10.622253,0.522300,0.767952,0.044722,8,complete,manhattan,"{'n_clusters': 8, 'linkage': 'complete', 'affi...",0.568282,0.501271,0.522853,0.529667,0.589062,0.542227,0.031884,7
1,9.731920,0.219575,0.679516,0.009429,7,average,l2,"{'n_clusters': 7, 'linkage': 'average', 'affin...",0.655422,0.653651,0.632386,0.654444,0.637662,0.646713,0.009705,3
2,10.070617,0.248395,0.735751,0.050442,7,complete,l1,"{'n_clusters': 7, 'linkage': 'complete', 'affi...",0.570617,0.568008,0.521122,0.557196,0.618592,0.567107,0.031231,6
3,3.017350,0.124097,0.455595,0.017866,3,single,cosine,"{'n_clusters': 3, 'linkage': 'single', 'affini...",-0.339807,-0.306178,-0.222252,-0.152953,-0.283724,-0.260983,0.066247,8
4,0.002811,0.000746,0.000000,0.000000,5,ward,l2,"{'n_clusters': 5, 'linkage': 'ward', 'affinity...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
5,0.002284,0.000394,0.000000,0.000000,3,ward,cosine,"{'n_clusters': 3, 'linkage': 'ward', 'affinity...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
6,1.572162,0.025139,0.354245,0.006285,8,single,l1,"{'n_clusters': 8, 'linkage': 'single', 'affini...",0.653547,0.638327,0.597007,0.596505,0.621180,0.621313,0.022515,5
7,10.042060,0.075324,0.725915,0.034782,2,complete,l1,"{'n_clusters': 2, 'linkage': 'complete', 'affi...",0.738654,0.728715,0.711142,0.715499,0.701859,0.719174,0.013025,1
8,1.604995,0.035663,0.347925,0.006126,4,single,manhattan,"{'n_clusters': 4, 'linkage': 'single', 'affini...",0.690006,0.646758,0.612741,0.625554,0.688048,0.652622,0.031655,2
9,10.585611,1.227802,0.821685,0.148252,2,ward,euclidean,"{'n_clusters': 2, 'linkage': 'ward', 'affinity...",0.660991,0.697196,0.570427,0.560713,0.645025,0.626870,0.052919,4


In [14]:
# n_clusters, linkage, affinity = 5, 'ward', 'l2'

In [20]:
print("score_max:", score_max, " n_clusters:", n_clusters, " linkage:", linkage, " affinity:", affinity)

score_max: 0.7191736249351095  n_clusters: 2  linkage: complete  affinity: l1


In [21]:
import time
start_time = time.time()
hierarchical = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage, affinity=affinity)

# Determine which clusters each data point belongs to:
clusters_hierarchical = hierarchical.fit_predict(hi.get_df()) + 1

# Add cluster labels to the original data
clusters_hierarchical = pd.DataFrame(clusters_hierarchical, columns=['cluster'])
df_reduced = pd.concat([hi.get_df(), clusters_hierarchical], axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

MemoryError: Unable to allocate 37.2 GiB for an array with shape (4994151711,) and data type float64

In [ ]:
cluster=df_reduced['cluster']
df_reduced.drop(labels=['cluster'], axis=1,inplace = True)

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(df_reduced)
principalComponents = pd.DataFrame(data=principalComponents, index=df_reduced.index, columns=['PC1', 'PC2'])
principalComponents['cluster'] = cluster

In [ ]:
plt.scatter(principalComponents['PC1'], principalComponents['PC2'], c=principalComponents['cluster'],cmap=viridis)
plt.show()

In [ ]:
cluster.value_counts()